In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import psycopg2.extras as extras
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
from dotenv import dotenv_values
from datetime import datetime

## Note # Real system , you need to get data from PostGrest # if no incident  it will be 0
## Schcule to send summary case at 00:01 and get count yester day to predict begining of the comming week

In [134]:
dateTimeCol='incident_datetime'
dateCol='issue_date'

today_date=datetime.now()

countCol='count_incident'

imported_from='2021-10-01'

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

# tf='W-SUN'
tf='D'


projectId='pongthorn'
dataset_id='SMartDW'
table_id = f"{projectId}.{dataset_id}.daily_incident"

In [135]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [136]:
sql=f"""
select Date({dateTimeCol}) as {dateCol},count(*) as {countCol}  from app_incident
where {dateTimeCol}>='{imported_from}'
group by   Date({dateTimeCol}) order by Date({dateTimeCol})
"""
print(sql)
df_incident=list_data(sql,None,get_postgres_conn())
df_incident[dateCol]=pd.to_datetime(df_incident[dateCol], format='%Y-%m-%d')

print(df_incident.info())
print(df_incident.head())
print(df_incident.tail())


select Date(incident_datetime) as issue_date,count(*) as count_incident  from app_incident
where incident_datetime>='2021-10-01'
group by   Date(incident_datetime) order by Date(incident_datetime)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   issue_date      543 non-null    datetime64[ns]
 1   count_incident  543 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 8.6 KB
None
  issue_date  count_incident
0 2021-10-04               1
1 2021-10-07               1
2 2021-10-11               1
3 2021-10-18               1
4 2021-10-19               1
    issue_date  count_incident
538 2023-07-04              13
539 2023-07-05               5
540 2023-07-06               5
541 2023-07-07               1
542 2023-12-02               1


In [137]:
from_date=df_incident[dateCol].min().strftime('%Y-%m-%d')
to_date=today_date.strftime('%Y-%m-%d')
# to_date=df_incident[dateCol].max().strftime('%Y-%m-%d')
period_index=pd.date_range(start=from_date,end=to_date, freq='D')
print(period_index)
dfDate=pd.DataFrame(index=period_index,columns=['date'])
dfDate.reset_index(drop=False,inplace=True)
dfDate['date']=dfDate['index']
dfDate.drop(columns=['index'],inplace=True)
print(dfDate.info())
print(dfDate.head())
print(dfDate.tail())

DatetimeIndex(['2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07',
               '2021-10-08', '2021-10-09', '2021-10-10', '2021-10-11',
               '2021-10-12', '2021-10-13',
               ...
               '2023-06-29', '2023-06-30', '2023-07-01', '2023-07-02',
               '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06',
               '2023-07-07', '2023-07-08'],
              dtype='datetime64[ns]', length=643, freq='D')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    643 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 5.1 KB
None
        date
0 2021-10-04
1 2021-10-05
2 2021-10-06
3 2021-10-07
4 2021-10-08
          date
638 2023-07-04
639 2023-07-05
640 2023-07-06
641 2023-07-07
642 2023-07-08


In [138]:
df=dfDate.merge(df_incident,how='left',left_on='date',right_on=dateCol)
df=df[['date',countCol]]
df.fillna(value=0,inplace=True)
# df=df.set_index('date')
print(df.info())
print(df.tail(10))

# if time frame is bigger that day
# dfXYZ=df.resample(tf).sum()
# print(dfXYZ.info())
# print(dfXYZ.tail(10))

# dfXYZ.to_csv('NoIncident.csv',index=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 0 to 642
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            643 non-null    datetime64[ns]
 1   count_incident  643 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 15.1 KB
None
          date  count_incident
633 2023-06-29             2.0
634 2023-06-30             5.0
635 2023-07-01             3.0
636 2023-07-02             1.0
637 2023-07-03             8.0
638 2023-07-04            13.0
639 2023-07-05             5.0
640 2023-07-06             5.0
641 2023-07-07             1.0
642 2023-07-08             0.0


In [140]:
client = bigquery.Client(project=projectId)
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

try:
    
    insertDataFrameToBQ(df)
except Exception as ex:
    raise ex


Total  643 Imported closed incident to bigquery successfully
